# Colab Setup  
> Make sure you configure notebook with GPU: Click Edit->notebook settings->hardware accelerator->GPU

> Uncomment the following cell after opening in Google colab. (Do not uncomment it in local setup.)  

<a target="_blank" href="https://colab.research.google.com/github/SEED-VT/FedDebug/blob/main/fault-localization/Reproduce_Figure4-Figure7.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



In [ ]:
# !pip install pytorch-lightning
# !pip install diskcache
# !pip install dotmap
# !pip install torch torchvision torchaudio
# !pip install matplotlib
# !git clone https://github.com/SEED-VT/FedDebug.git
# # appending the path
# import sys
# sys.path.append("FedDebug/fault-localization/")

# Descripiton

- The code is simulating the presence of a faulty client in a federated learning setting by introducing noise to the model updates from a faulty client. The `noise_rate` variable is used to control the amount of noise added.

- For each noise level, the code performs FL training, aggregation to update the global model, and testing of the global model's accuracy. 

- The code also runs `FedDebug's fault localization algorithm` to identify potentially faulty clients and evaluate the accuracy of the fault localization. 

- The results of the simulation are stored in a list, and the `global model accuracy` and `fault localization accuracy` are plotted against the `noise rate`.


In [ ]:
import logging
import matplotlib.pyplot as plt
import time
from dotmap import DotMap
from pytorch_lightning import seed_everything
from torch.nn.init import kaiming_uniform_ 
from utils.faulty_client_localization.FaultyClientLocalization import FaultyClientLocalization
from utils.faulty_client_localization.InferenceGuidedInputs import InferenceGuidedInputs
from utils.FLSimulation import trainFLMain
from utils.fl_datasets import initializeTrainAndValidationDataset
from utils.util import aggToUpdateGlobalModel
from utils.util import testAccModel



logging.basicConfig(filename='example.log', level=logging.ERROR)
logger = logging.getLogger("pytorch_lightning")
seed_everything(786)



def evaluateFaultLocalization(predicted_faulty_clients_on_each_input, true_faulty_clients):
    true_faulty_clients = set(true_faulty_clients)
    detection_acc = 0
    for pred_faulty_clients in predicted_faulty_clients_on_each_input:
        print(f"+++ Faulty Clients {pred_faulty_clients}")
        correct_localize_faults = len(
            true_faulty_clients.intersection(pred_faulty_clients))
        acc = (correct_localize_faults/len(true_faulty_clients))*100
        detection_acc += acc
    fault_localization_acc = detection_acc / \
        len(predicted_faulty_clients_on_each_input)
    return fault_localization_acc


def runFaultyClientLocalization(client2models, exp2info, num_bugs, random_generator=kaiming_uniform_, apply_transform=True, k_gen_inputs=10, na_threshold=0.003, use_gpu=True):
    print(">  Running FaultyClientLocalization ..")
    input_shape = list(exp2info['data_config']['single_input_shape'])
    generate_inputs = InferenceGuidedInputs(client2models, input_shape, randomGenerator=random_generator, apply_transform=apply_transform,
                                            dname=exp2info['data_config']['name'], min_nclients_same_pred=5, k_gen_inputs=k_gen_inputs)
    selected_inputs, input_gen_time = generate_inputs.getInputs()

    start = time.time()
    faultyclientlocalization = FaultyClientLocalization(
        client2models, selected_inputs, use_gpu=use_gpu)

    potential_benign_clients_for_each_input = faultyclientlocalization.runFaultLocalization(
        na_threshold, num_bugs=num_bugs)
    fault_localization_time = time.time()-start
    return potential_benign_clients_for_each_input, input_gen_time, fault_localization_time





# ====== Simulation ===== 

args = DotMap()
args.lr = 0.001
args.weight_decay = 0.0001
args.batch_size = 512

nosie2results = []
different_noise_levels = [0.1, 0.3, 0.5, 0.7, 0.9, 1]
args.model = "resnet18" # [resnet18, resnet34, resnet50, densenet121, vgg16]
args.epochs = 5  # range 10-25
args.dataset = "cifar10" # ['cifar10', 'femnist']
args.clients = 10 # keep under 30 clients and use Resnet18, Resnet34, or Densenet to evaluate on Colab 
args.faulty_clients_ids = "0" # can be multiple clients separated by comma e.g. "0,1,2"  but keep under args.clients clients and at max less than 7 
args.sampling = "iid" # [iid, "niid"] 

_,test_data, _ =  initializeTrainAndValidationDataset(args.dataset, ".storage/datasets/")

for noise_rate in different_noise_levels:
    args.noise_rate = noise_rate  # noise rate 0 to 1 
    # FL training
    c2ms, exp2info = trainFLMain(args)
    client2models = {k: v.model.eval() for k, v in c2ms.items()}

    gm =  aggToUpdateGlobalModel(clients_models=client2models)
    test_acc = testAccModel(gm, test_data)

    # Fault localazation
    potential_faulty_clients, _, _ = runFaultyClientLocalization(
        client2models=client2models, exp2info=exp2info, num_bugs=len(exp2info['faulty_clients_ids']))
    fault_acc = evaluateFaultLocalization(
        potential_faulty_clients, exp2info['faulty_clients_ids'])
    print(f"Fault Localization Acc: {fault_acc}")

    print(f"Gloabl Model Test Acc: {test_acc}, Noise Rate: {noise_rate}, Fault Acc: {fault_acc}")
    nosie2results.append((noise_rate, test_acc, fault_acc))



In [ ]:
x =different_noise_levels 
y1_test_acc = [y[1] for y in nosie2results]
y2_fault_acc = [y[2] for y in nosie2results]
fig, axes = plt.subplots(1, 2) 
axes[0].plot(x, y1_test_acc, 'g--o') 
axes[1].plot(x, y2_fault_acc, 'm--o')

axes[0].set_xlabel('Noise Rate')
axes[1].set_xlabel('Noise Rate')
axes[0].set_ylabel('Global Model Accuracy (%)')
axes[1].set_ylabel('Fault Localization Accuracy (%)')
axes[0].set_title("Paper Figure 4 (a)")
axes[1].set_title("Paper Figure 7 (a)")

axes[0].set_ylim(0, 105)
axes[1].set_ylim(0, 105)

fig.set_figwidth(10)

plt.tight_layout()
